In [ ]:
# Second test for git aliases (e.g., git-update-plus, git-next-checkin, etc)

# Note: The aliases should be defined before Jupyter is invoked.
# For BatsPP, they can loaded via '--source all-tomohara-aliases-etc.bash' option.

In [ ]:
# Global Setup
# Creating temp local repos to test on
TMPA=/tmp/temp-git-test

In [ ]:
# Global setup
# Define aliases for encoding output (e.g., to ensure output the same across runs).
alias encode-control-chars='perl -pe "s/\t/<tab>/g;  s/[\x00-\x08\x11\x31]/<cc>/g;"'
alias encode-nums='perl -pe "s/[0-9]+/N/g;"'
## TODO: define a better hex encoder to avoid false positives (e.g., abc).
alias encode-hex='perl -pe "s/[0-9A-F]{4,}/<hex>/ig;"'
# note: uses perl slurp mode (i.e., read entire file) to check for consecutive newlines
alias encode-newlines='perl -0777 -pe "while (/\n\n/) { s/\n\n/\n<newline>\n/g; }"'
function encode-text () { encode-control-chars | encode-nums | encode-hex | encode-newlines; }

In [ ]:
# Global setup
git clone https://github.com/tomasohara/git-bash-test.git "$TMPA" > /dev/null 2>&1

In [ ]:
# Setup
# Move to second local to push some changes
command cd "$TMPA" > /dev/null 2>&1
git pull origin main > /dev/null 2>&1
echo "hello second local" > hello.txt
git add hello.txt > /dev/null 2>&1
git commit -m "first commit on second local" > /dev/null 2>&1
git push > /dev/null 2>&1

In [ ]:
# git-rename-file tests
# Assert file doesn't exist
[ ! -f "newHello.txt" ]; echo $?;

In [ ]:
# Setup
export GIT_MESSAGE="renamed hello.txt"
previous=$GIT_NO_CONFIRM
export GIT_NO_CONFIRM="1"
contents="$(cat hello.txt)"
git-rename-file hello.txt newHello.txt > /dev/null 2>&1 

In [ ]:
# Asserts
[ -f "newHello.txt" ] && [ "$(cat newHello.txt)" == "$(echo $contents)" ]; echo $?;

In [ ]:
# Setup
# git-restore-staged-alias test
echo "second change on local" >> newHello.txt
git add newHello.txt > /dev/null 2>&1

In [ ]:
# Assert theres is one staged file 
[ $(git status -s | grep -c "^[MTADRCU]") -eq 1 ]; echo $?; 

In [ ]:
# Setup
export GIT_MESSAGE="restored newHello.txt"
export GIT_NO_CONFIRM="1"
git-restore-staged-alias newHello.txt > /dev/null 2>&1
rm -rf _git-trash > /dev/null

In [ ]:
# Assert theres are no staged files
[ $(git status -s | grep -c "^[MTADRCU]") -eq 0 ]; echo $?; 

In [ ]:
# Setup
# git-revert-file-alias tests
echo "second change on local" >> newHello.txt

In [ ]:
# Assert theres is 1 unstaged file
[ $(git status -s | grep -c "^\s[MTADRCU]") -eq 1 ]; echo $?; 

In [ ]:
# Setup
# revert unstaged file
export GIT_MESSAGE="reverted newHello.txt"
export GIT_NO_CONFIRM="1"
git-revert-file-alias newHello.txt > /dev/null 2>&1
rm -rf _git-trash > /dev/null 2>&1

In [ ]:
# Assert file was reverted
# this matches any files except untracked
# read https://git-scm.com/docs/git-status#_short_format
[ $(git status -s | grep -c "^\s*[MTADRCU]") -eq 0 ]; echo $?; 

In [ ]:
# Setup
# git-move-to-dir tests
command mkdir folder
git add folder > /dev/null 2>&1
git-move-to-dir "$TMP/folder" newHello.txt > /dev/null 2>&1 | true 
command cd folder

In [ ]:
# make sure the file was moved (at least one file change)
[ $(ls | wc -l) -eq 1 ]; echo $?;

In [ ]:
# Setup
# git-conflicts-alias tests
command cd $TMPA

In [ ]:
# assert there are no conflicts already
[ $(git-conflicts-alias | wc -l) -eq 0 ]; echo $?

In [ ]:
# Setup
# create branches that conflict later
git branch branch-a > /dev/null 2>&1
git branch branch-b > /dev/null 2>&1

In [ ]:
# Setup
git checkout branch-a >/dev/null 2>&1
echo "line a" >> conflicts.txt
git add . > /dev/null 2>&1
git commit -m "first commit branch-a" > /dev/null 2>&1

In [ ]:
# Setup
git switch branch-b
echo "line b" >> conflicts.txt
git add . > /dev/null 2>&1
git commit -m "first commit branch-b" > /dev/null 2>&1

In [ ]:
# Setup
# merge branches 
git switch main > /dev/null 2>&1
git merge branch-a > /dev/null 2>&1
git merge branch-b | true > /dev/null 2>&1

In [ ]:
# assert some conflicts were generated
[ $(git-conflicts-alias | wc -l) -gt 0 ]; echo $?;

In [ ]:
# Setup
# undo changes and remove newly created files
rm -f newHello.txt conflicts.txt > /dev/null 2>&1
git rm -f *.txt > /dev/null 2>&1
git rm -rf folder > /dev/null 2>&1
git add . > /dev/null 2>&1

In [ ]:
# Setup
## Lorenzo-TODO2: use one cell per output (to make this less brittle)
git commit -m "cleanup test files" > /dev/null 2>&1
git push > /dev/null 2>&1

In [ ]:
# Setup
rename-with-file-date /tmp/temp-git-test > /dev/null
export GIT_NO_CONFIRM=$previous